In [18]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


### Data Science Level 2 Practical Problem 2305-6

In [3]:
df = pd.read_csv('lv2-2305-6.csv')

#### Question 1

In [4]:
df1 = df.copy()

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

sc = MinMaxScaler(feature_range=(2.2,5.7))
# Step 1-1
A = sc.fit_transform(df1[['SURVEY_RES_01']]).std()
m = 7.9
std = 1.399
df1['SURVEY_RES_02'] = (df1['SURVEY_RES_02']*std + m)
B = df1['SURVEY_RES_02'].median()
ans21 = A + B

In [7]:
print('Q1 Result:', ans21)
print('Q1 Answer:', round(ans21,2))

Q1 Result: 8.91329731865739
Q1 Answer: 8.91


#### Question 2

In [8]:
df2 = df.copy()

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [10]:
# Step 2-1
df2_1 = df2.iloc[:,1:-1]
df2_1['const'] = 1

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(df2_1.values, i) for i in range(df2_1.shape[1])]

vif['features'] = df2_1.columns

vif = vif[vif['features'] != 'const']
if vif['VIF Factor'].max() >= 5:
    df2_1 = df2_1.drop(columns = [vif[vif['VIF Factor'] == vif['VIF Factor'].max()]['features'].values[0]])
df2_1 = df2_1.drop(columns = ['const'])

In [11]:
df2_1['SATISFACTION'] = df2['SATISFACTION']
df2_1['CUST_ID'] = df2['CUST_ID']

In [19]:
# Step 2-2 
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = 'newton-cg', C=100000, random_state=1234)


train = df2_1[df2_1['CUST_ID'] <= 1000]
test = df2_1[df2_1['CUST_ID'] > 1000]

train_x = train.drop(columns = ['CUST_ID', 'SATISFACTION'])
train_y = train[['SATISFACTION']]

test_x = test.drop(columns = ['CUST_ID', 'SATISFACTION'])
test_y = test[['SATISFACTION']]

lr.fit(train_x, train_y)

test_y['prd'] = lr.predict(test_x)
from sklearn.metrics import accuracy_score
C = accuracy_score(test_y['SATISFACTION'], test_y['prd'])


In [20]:
# Step 2-3
tmp = 0
res = []
for i in range(6):
    lb = tmp
    ub = tmp+200
    test = df2_1[(df2_1['CUST_ID'] > lb) & (df2_1['CUST_ID'] <= ub)]
    train = df2_1[((df2_1['CUST_ID'] > 0) & (df2_1['CUST_ID'] <=lb)) | ((df2_1['CUST_ID'] > ub) & (df2_1['CUST_ID'] <= 1200))]    
    train_x = train.drop(columns = ['CUST_ID', 'SATISFACTION'])
    train_y = train[['SATISFACTION']]
    
    test_x = test.drop(columns = ['CUST_ID', 'SATISFACTION'])
    test_y = test[['SATISFACTION']]
    
    lr = LogisticRegression(solver = 'newton-cg', C=100000, random_state=1234)
    lr.fit(train_x, train_y)
#     print(test['CUST_ID'].min(), test['CUST_ID'].max())
    test_y['prd'] = lr.predict(test_x)
    res.append(accuracy_score(test_y['SATISFACTION'], test_y['prd']))
    tmp = tmp + 200
D = sum(res)/6

In [15]:
ans22 = abs(C - D)
print('Q2 Result:', ans22)
print('Q2 Answer:', int(ans22 * 1000) / 1000)

Q2 Result: 0.012499999999999956
Q2 Answer: 0.012


#### Question 3

In [16]:
df3 = df.copy()

# Step 3-1
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
sc.fit(df3.iloc[:,2:-1])
list_cols = df3.iloc[:,2:-1].columns.tolist()
df3_1 = pd.concat([df3, \
                   pd.DataFrame(sc.transform(df3.iloc[:,2:-1]), \
                                columns = [x + '_N' for x in list_cols])], axis=1)

# Step 3-2
df3_1.rename(columns = {'SURVEY_RES_01': 'SURVEY_RES_01_N'},inplace=True)

# Step 3-3 
from sklearn.cluster import KMeans

k_list = [3,5,7]

df3_2 = df3_1.drop(columns = ['CUST_ID', 'SATISFACTION'])
df3_3 = df3_2.drop(columns = list_cols)

from sklearn.metrics import silhouette_samples, silhouette_score

max_num = -999
res_list = []
i = 0
for k in k_list:
    md = KMeans(n_clusters=k, random_state = 1234, max_iter = 300, n_init = 50)
    md.fit(df3_3)
    res = md.predict(df3_3)
    res_list.append(res)
    score = silhouette_score(df3_3, res)
    if score > max_num:
        max_num = score
        max_k = k
        max_i = i
    i = i + 1

tot = 0
for num in range(k):
    num_c = np.count_nonzero(res_list[max_i] == num)
    if tot < num_c:
        tot = num_c

ans23 = tot

In [17]:
print('Q3 Answer: ', ans23)

Q3 Answer:  261
